In [2]:
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as f
findspark.init()

# Creating a SparkSession in Python
spark = SparkSession.builder\
    .master("local")\
    .appName("Spark Streaming")\
    .getOrCreate()

# keep the size of shuffles small
spark.conf.set("spark.sql.shuffle.partitions", "2")

your 131072x1 screen size is bogus. expect trouble
24/05/14 16:35:12 WARN Utils: Your hostname, PhDDevice resolves to a loopback address: 127.0.1.1; using 172.24.86.99 instead (on interface eth0)
24/05/14 16:35:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/14 16:35:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
full_schema = StructType([StructField(f"_c{i}", StringType(), nullable=True) for i in range(22)])

# define schema for yellow taxi trips
yellow_taxi_schema = StructType([
    StructField("type", StringType(), nullable=False),
    StructField("VendorID", IntegerType()),
    StructField("tpep_pickup_datetime", TimestampType()),
    StructField("tpep_dropoff_datetime", TimestampType()),
    StructField("passenger_count", IntegerType()),
    StructField("trip_distance", FloatType()),
    StructField("pickup_longitude", FloatType()),
    StructField("pickup_latitude", FloatType()),
    StructField("RatecodeID", FloatType()),
    StructField("store_and_fwd_flag", StringType()),
    StructField("dropoff_longitude", FloatType()),
    StructField("dropoff_latitude", FloatType()),
    StructField("payment_type", IntegerType()),
    StructField("fare_amount", FloatType()),
    StructField("extra", FloatType()),
    StructField("mta_tax", FloatType()),
    StructField("tip_amount", FloatType()),
    StructField("tolls_amount", FloatType()),
    StructField("improvement_surcharge", FloatType()),
    StructField("total_amount", FloatType())
])
# define schema for green taxi trips
green_taxi_schema = StructType([
    StructField("type", StringType(), nullable=False),
    StructField("VendorID", IntegerType()),
    StructField("lpep_pickup_datetime", TimestampType()),
    StructField("Lpep_dropoff_datetime", TimestampType()),
    StructField("Store_and_fwd_flag", StringType()),
    StructField("RateCodeID", IntegerType()),
    StructField("Pickup_longitude", FloatType()),
    StructField("Pickup_latitude", FloatType()),
    StructField("Dropoff_longitude", FloatType()),
    StructField("Dropoff_latitude", FloatType()),
    StructField("Passenger_count", IntegerType()),
    StructField("Trip_distance", FloatType()),
    StructField("Fare_amount", FloatType()),
    StructField("Extra", FloatType()),
    StructField("MTA_tax", FloatType()),
    StructField("Tip_amount", FloatType()),
    StructField("Tolls_amount", FloatType()),
    StructField("Ehail_fee", FloatType()),
    StructField("improvement_surcharge", FloatType()),
    StructField("Total_amount", FloatType()),
    StructField("Payment_type", IntegerType()),
    StructField("Trip_type", IntegerType())
])

In [4]:
# read the entire data files first
full_df = spark.readStream\
    .option("maxFilesPerTrigger", 1)\
    .csv(f"file:///home/dat_21127240/BigData/Big-Data-With-Seaborn-Bokeh-Plotly/Lab_03/data/taxi-data",
         header=False,
         schema=full_schema)

# Filter rows based on the 'type' column
# yellow_df = full_df.where(full_df["_c0"] == "yellow").selectExpr(
#     *[f"cast(_c{i} as {field.dataType.simpleString()}) as {field.name}" \
#         for i, field in enumerate(yellow_taxi_schema.fields)]
# )
# green_df = full_df.where(full_df["_c0"] == "green").selectExpr(
#     *[f"cast(_c{i} as {field.dataType.simpleString()}) as {field.name}" \
#         for i, field in enumerate(green_taxi_schema.fields)]
# )

In [ ]:
# Query
full_df = full_df.withColumn("dropoff", f.expr("cast(_c3 as timestamp)"))
by_dropoff = full_df\
    .withWatermark("dropoff", "1 hour")\
    .groupBy(f.window(f.col("dropoff"), "1 hour"))\
    .count()

# Function to handle each batch
def process_row(row: Row):
    # # Extract the start time of the window
    # window_end = batch_df.selectExpr("window.end").first()["end"]
    # # Convert to timestamp in milliseconds
    # timestamp_ms = window_end.hour * 60 * 60 * 1000
    # # Create output directory name
    # output_dir = f"file:///home/dat_21127240/BigData/Big-Data-With-Seaborn-Bokeh-Plotly/output/output-{timestamp_ms}"
    
    # # Write the batch DataFrame to the HDFS directory
    # batch_df.write.mode("overwrite").json(output_dir)
    print(row)

query = by_dropoff.writeStream\
    .outputMode("complete")\
    .foreach(process_row)\
    .queryName("Event_Count")\
    .option("truncate", "false")\
    .start()\
    .awaitTermination(10)

Row(window=Row(start=datetime.datetime(2015, 12, 1, 10, 0), end=datetime.datetime(2015, 12, 1, 11, 0)), count=1115)
Row(window=Row(start=datetime.datetime(2015, 12, 1, 11, 0), end=datetime.datetime(2015, 12, 1, 12, 0)), count=1398)
Row(window=Row(start=datetime.datetime(2015, 12, 1, 7, 0), end=datetime.datetime(2015, 12, 1, 8, 0)), count=377)
Row(window=Row(start=datetime.datetime(2015, 12, 1, 2, 0), end=datetime.datetime(2015, 12, 1, 3, 0)), count=52)
Row(window=Row(start=datetime.datetime(2015, 12, 1, 3, 0), end=datetime.datetime(2015, 12, 1, 4, 0)), count=94)
Row(window=Row(start=datetime.datetime(2015, 12, 1, 4, 0), end=datetime.datetime(2015, 12, 1, 5, 0)), count=93)
Row(window=Row(start=datetime.datetime(2015, 12, 1, 8, 0), end=datetime.datetime(2015, 12, 1, 9, 0)), count=1939)
Row(window=Row(start=datetime.datetime(2015, 12, 1, 1, 0), end=datetime.datetime(2015, 12, 1, 2, 0)), count=496)
Row(window=Row(start=datetime.datetime(2015, 12, 1, 5, 0), end=datetime.datetime(2015, 12, 1